In [115]:
import sys
assert sys.version_info >= (3, 5)
# Python ≥3.5 is required
import sklearn
assert sklearn.__version__ >= "0.20"
import numpy as np
import os
import tarfile
import urllib
import pandas as pd
import urllib.request
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn import preprocessing
from sklearn import metrics
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
from pathlib import Path

In [116]:
data = pd.read_csv("02-19-2023.csv")

In [117]:
geoJSONDict = {

  "type": "FeatureCollection",
  "crs": {
    "type": "name",
    "properties": {
      "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
    }
  },
  "features": []
}


In [118]:
# incident rate = (Cases/population)*100,000
# population = (Cases*100,000)/incident rate
us_data = data.loc[data['Country_Region']=='US']
data = data[data['Country_Region']!='US']
data["Population"] = (data["Confirmed"] * 100000)/data["Incident_Rate"]
data["Case_Fatality_Ratio"] = (data["Case_Fatality_Ratio"]*100).round()
data.head(50)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Population
0,NaN,NaN,NaN,Afghanistan,2023-02-20 04:21:05,33.939110,67.709953,209107,7896,NaN,NaN,Afghanistan,537.158776,378.0,38928341.0
1,NaN,NaN,NaN,Albania,2023-02-20 04:21:05,41.153300,20.168300,334314,3596,NaN,NaN,Albania,11616.999097,108.0,2877800.0
2,NaN,NaN,NaN,Algeria,2023-02-20 04:21:05,28.033900,1.659600,271425,6881,NaN,NaN,Algeria,618.970454,254.0,43851043.0
3,NaN,NaN,NaN,Andorra,2023-02-20 04:21:05,42.506300,1.521800,47866,165,NaN,NaN,Andorra,61950.430337,34.0,77265.0
4,NaN,NaN,NaN,Angola,2023-02-20 04:21:05,-11.202700,17.873900,105184,1931,NaN,NaN,Angola,320.036336,184.0,32866268.0
5,NaN,NaN,NaN,Antarctica,2023-02-20 04:21:05,-71.949900,23.347000,11,0,NaN,NaN,Antarctica,NaN,0.0,NaN
6,NaN,NaN,NaN,Antigua and Barbuda,2023-02-20 04:21:05,17.060800,-61.796400,9106,146,NaN,NaN,Antigua and Barbuda,9298.668409,160.0,97928.0
7,NaN,NaN,NaN,Argentina,2023-02-20 04:21:05,-38.416100,-63.616700,10042136,130448,NaN,NaN,Argentina,22219.190966,130.0,45195777.0
8,NaN,NaN,NaN,Armenia,2023-02-20 04:21:05,40.069100,45.038200,446332,8719,NaN,NaN,Armenia,15062.327174,195.0,2963234.0
9,NaN,NaN,Australian Capital Territory,Australia,2023-02-20 04:21:05,-35.473500,149.012400,231529,224,NaN,NaN,"Australian Capital Territory, Australia",54082.924550,10.0,428100.0


In [119]:
us_data

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
678,1001.0,Autauga,Alabama,US,2023-02-20 04:21:05,32.539527,-86.644082,19686,230,NaN,NaN,"Autauga, Alabama, US",35235.998496,1.168343
679,1003.0,Baldwin,Alabama,US,2023-02-20 04:21:05,30.727750,-87.722071,69521,724,NaN,NaN,"Baldwin, Alabama, US",31142.657480,1.041412
680,1005.0,Barbour,Alabama,US,2023-02-20 04:21:05,31.868263,-85.387129,7431,103,NaN,NaN,"Barbour, Alabama, US",30102.082152,1.386085
681,1007.0,Bibb,Alabama,US,2023-02-20 04:21:05,32.996421,-87.125115,8049,109,NaN,NaN,"Bibb, Alabama, US",35942.663213,1.354205
682,1009.0,Blount,Alabama,US,2023-02-20 04:21:05,33.982109,-86.567906,18554,261,NaN,NaN,"Blount, Alabama, US",32085.912911,1.406705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,56039.0,Teton,Wyoming,US,2023-02-20 04:21:05,43.935225,-110.589080,12113,16,NaN,NaN,"Teton, Wyoming, US",51623.764064,0.132089
3953,56041.0,Uinta,Wyoming,US,2023-02-20 04:21:05,41.287818,-110.547578,6373,43,NaN,NaN,"Uinta, Wyoming, US",31508.948878,0.674721
3954,90056.0,Unassigned,Wyoming,US,2023-01-08 23:21:00,NaN,NaN,0,0,NaN,NaN,"Unassigned, Wyoming, US",NaN,NaN
3955,56043.0,Washakie,Wyoming,US,2023-02-20 04:21:05,43.904516,-107.680187,2744,50,NaN,NaN,"Washakie, Wyoming, US",35156.950673,1.822157


In [120]:
#data struct to store states
# for every state in us_data: 
# get average of Case_Fatality_Ratio, median of lat and long, country
# insert row into data

states =  us_data.Province_State.values.tolist()
states = list(dict.fromkeys(states))

for state in states:
    tempDf = us_data.loc[us_data['Province_State']==state]
    stateDf = tempDf.groupby('Province_State', as_index=False).Case_Fatality_Ratio.max()
    lat = tempDf.groupby('Province_State', as_index=False).Lat.median().iloc[0]['Lat']
    long = tempDf.groupby('Province_State', as_index=False).Long_.median().iloc[0]['Long_']
    country = "US"
    stateDf["Lat"] = lat
    stateDf["Long_"] = long
    stateDf["Country_Region"] = country
    stateDf
    data = data.append(stateDf, ignore_index=True)

C:\Users\Shawn\AppData\Local\Temp\ipykernel_20736\1243832962.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_20736\1243832962.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_20736\1243832962.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_20736\1243832962.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)


C:\Users\Shawn\AppData\Local\Temp\ipykernel_20736\1243832962.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_20736\1243832962.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_20736\1243832962.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_20736\1243832962.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)


In [121]:
sampleDf = data.loc[data['Country_Region']=='US']
sampleDf

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Population
737,NaN,NaN,Alabama,US,NaN,32.855042,-86.662326,NaN,NaN,NaN,NaN,NaN,NaN,2.388060,NaN
738,NaN,NaN,Alaska,US,NaN,60.067639,-149.953805,NaN,NaN,NaN,NaN,NaN,NaN,0.867679,NaN
739,NaN,NaN,American Samoa,US,NaN,-14.271000,-170.132000,NaN,NaN,NaN,NaN,NaN,NaN,0.408654,NaN
740,NaN,NaN,Arizona,US,NaN,33.348359,-111.344948,NaN,NaN,NaN,NaN,NaN,NaN,2.663969,NaN
741,NaN,NaN,Arkansas,US,NaN,35.002924,-92.499744,NaN,NaN,NaN,NaN,NaN,NaN,3.789593,NaN
742,NaN,NaN,California,US,NaN,38.048832,-120.918298,NaN,NaN,NaN,NaN,NaN,NaN,1.545284,NaN
743,NaN,NaN,Colorado,US,NaN,38.934415,-105.451273,NaN,NaN,NaN,NaN,NaN,NaN,8.984375,NaN
744,NaN,NaN,Connecticut,US,NaN,41.641359,-72.634094,NaN,NaN,NaN,NaN,NaN,NaN,1.462510,NaN
745,NaN,NaN,Delaware,US,NaN,39.086466,-75.568849,NaN,NaN,NaN,NaN,NaN,NaN,1.268488,NaN
746,NaN,NaN,Diamond Princess,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN


In [122]:
clean_data = data.drop(['FIPS', 'Admin2','Province_State','Country_Region','Last_Update','Deaths','Recovered','Active','Combined_Key','Incident_Rate','Confirmed','Population'], axis=1)
clean_data = clean_data.dropna()
clean_data.loc[~(clean_data==0.0).all(axis=1)]

,Lat,Long_,Case_Fatality_Ratio
0,33.939110,67.709953,378.000000
1,41.153300,20.168300,108.000000
2,28.033900,1.659600,254.000000
3,42.506300,1.521800,34.000000
4,-11.202700,17.873900,184.000000
...,...,...,...
791,37.375699,-78.173251,3.454545
792,47.140037,-120.791360,2.409639
793,38.892851,-80.706715,1.991885
794,44.434222,-89.668273,2.773156


In [123]:
clean_data["Case_Fatality_Ratio"].sum()

174765.6397587355

In [124]:
#The mistake made here is Im rendering EACH AND EVERY covid case 
#Instead of that, create an object with long, lat and total case count.
#Hence we should only have approx 4000 json objects instead of 600 million+
#TODO: Figure out how mapbox is gonna render it



for i in range(len(clean_data)):
    
    longitude = clean_data.iloc[i]['Long_']
    latitude = clean_data.iloc[i]['Lat']
    data_points = int(clean_data.iloc[i]['Case_Fatality_Ratio'])
    
    
    for x in range(data_points):
        geometry = {
        "geometry": {
          "coordinates": [
            longitude,
            latitude
          ]
        }
      }
        geoJSONDict["features"].append(geometry)
    

json_object = json.dumps(geoJSONDict, separators=(',', ":"))
with open("../server/case_fatality_ratio.json", "w") as outfile:
    outfile.write(json_object)
    

In [ ]:
csv_total